In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from collections import Counter
from sklearn.model_selection import train_test_split

## 1. Reading input datasets

### 1.1 Reading the text data

In [5]:
text_data_sentence = pd.read_csv('./Files/textdatanew.csv', encoding='ISO-8859-1')
text_data_sentence.head(5)

,CollectionID,BiographyID,ParagraphNo,ParagraphText
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ..."
1,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...
2,a001,bio04,3,Cleopatra might have responded with a brillian...
3,a001,bio04,4,Caesar was then above fifty years of age. His ...
4,a001,bio04,5,For three years Cleopatra reigned with little ...


### 1.2 Reading the text features

In [6]:
text_features = pd.read_csv("text_features.csv", encoding='ISO-8859-1')
text_features.head(2)

C:\Users\arvra\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (11,12,15,16,22,23,24,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CollectionID,BiographyID,ParagraphNo,sadness,joy,fear,disgust,anger,score,sentiment,...,Number,Organization,Person,PrintMedia,Quantity,Sport,SportingEvent,TelevisionShow,Time,Vehicle
0,a001,bio04,1,0.255896,0.558011,0.101166,0.111615,0.054668,0.290669,positive,...,NaN,NaN,Cleopatra,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a001,bio04,2,0.171629,0.257088,0.173474,0.098726,0.267978,0.000000,neutral,...,NaN,Roman senate,"Cleopatra, Julius Caesar, Pompey, Ptolemy",NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.3 Reading the Response file

In [8]:
bess_tags = pd.read_csv('CBW_Bess_tags_final2.csv')
bess_tags.head()

,Content,Event,Type,para no,URI,author,biographyID,collectionID,personaName,title
0,after,name,stageOfLife,1.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
1,culmination,name,stageOfLife,1.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
2,middle,name,stageOfLife,2.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
3,middle,name,stageOfLife,3.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
4,middle,name,stageOfLife,4.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...


### 2. Preprocessing Data

### 2.1 Getting the top Event types

In [9]:
bess_reponse = bess_tags.loc[:,['Content','Event','Type','para no','biographyID','collectionID']]

bess_reponse= bess_reponse.fillna(' ')

bess_reponse.loc[:,'Response'] = bess_reponse.loc[:,['Content','Event']].apply(lambda x: '_'.join(x),axis = 1)

bess_reponse['Bio_col_id'] = bess_reponse['biographyID'] +"_" + bess_reponse['collectionID']
bess_reponse['Bio_col_para_id'] = bess_reponse['Bio_col_id'] +"_" + bess_reponse['para no'].astype('str')

doc_count = pd.DataFrame(bess_reponse[bess_reponse.Type.isin(['Event'])].\
                         groupby(['Response'])['Bio_col_id'].apply(lambda x: len(np.unique(x))))

#############################################################################
##########TF - IDF Approach to get the top event types ######################
#############################################################################

term_freq = pd.DataFrame(bess_reponse[bess_reponse.Type.isin(['Event'])].\
                            groupby(['Response'])['Bio_col_id'].count())

total_docs = len(bess_reponse['Bio_col_id'].unique())

In [106]:
group_by_counts = pd.concat([term_freq,doc_count],axis = 1)

group_by_counts.columns = ['Term_freq','Doc_freq']
group_by_counts['tf_idf'] = pd.DataFrame(group_by_counts['Term_freq'] * np.log(total_docs/group_by_counts['Doc_freq']) )

group_by_counts.sort_values(['tf_idf'],ascending=False)[0:20]

,Term_freq,Doc_freq,tf_idf
Response,,,
"lover, male, named_agentType",776,75,1091.131263
hospital_locationStructure,617,71,901.378524
"sovereign, male_agentType",655,79,886.959898
"nobleman, named_agentType",853,122,784.388141
husband_agentType,1245,165,768.961337
"royalty, male_agentType",753,111,763.583340
conversation_type,1118,171,650.588288
"nursing, professional_type",281,33,625.808789
famous man_agentType,712,129,615.006161


In [11]:
# bio_response = pd.DataFrame(bess_reponse.groupby(['Response'])['Bio_col_para_id'].apply(lambda x: len(np.unique(x))))
# bio_response.sort_values(['Bio_col_para_id'],ascending=False).head()

### 2.2 Creating the Respone Variable for the model

In [107]:
### Select the event for building the model
reponse_required = 'lover, male, named_agentType'

In [108]:
reponse_required_to_merge = bess_reponse[bess_reponse.Response == reponse_required]

##### Merging the features and the response dataset
text_data_merge = pd.merge(text_data_sentence, reponse_required_to_merge.drop_duplicates(),\
                     how = 'left', left_on=['CollectionID','BiographyID','ParagraphNo'],
                         right_on=['collectionID','biographyID','para no'])


########## Final Data Frame #############
final_data_frame = text_data_merge.loc[:,['ParagraphText','Response']]

final_data_frame['Response_binary'] = np.where(final_data_frame.Response.isnull(),0,1)

##### Distribution of the Response variable

In [109]:
final_data_frame.Response_binary.value_counts()

0    16054
1      583
Name: Response_binary, dtype: int64

### 2.3 Prepocessing the Paragraph Text

##### 2.3.1 StopWord collection

#### Getting stop words - High Frequency and Low Frequency word list

In [110]:
from collections import Counter
from nltk.tokenize import word_tokenize

tokenized_para = final_data_frame.ParagraphText.apply(word_tokenize)

all_sent = [words for each_sent in tokenized_para for words in each_sent]

count_dict = Counter(all_sent)

high_freq_words = [word for (word,count) in count_dict.most_common(500)]

less_freq_words = []
threshold = 5

for k,v in count_dict.items():
    
    if v < threshold:
        less_freq_words.append(k)

        
######### List of all stop words ##########

stop_words = stopwords.words('english')
stop_words.extend(high_freq_words)
stop_words.extend(less_freq_words)

### 3. Model Building

#### 3.1 Creating Training and Test datasets

In [111]:
X_train, X_test, y_train, y_test = train_test_split(final_data_frame.ParagraphText ,final_data_frame.Response_binary,
                                                    test_size = 0.2, random_state = 0)

#### 3.2 Splitting the dataset into two categories

In [112]:
data_bin_1 = X_train[y_train == 1]
data_bin_0 = X_train[y_train == 0]

dict_bin_1_tokens = word_tokenize(' '.join(data_bin_1))
dict_bin_0_tokens = word_tokenize(' '.join(data_bin_0))

dictionary_bin_1 = Counter([each for each in dict_bin_1_tokens if each not in stop_words])
dictionary_bin_0 = Counter([each for each in dict_bin_0_tokens if each not in stop_words])

In [113]:
################ Creating a dataframe of probabilites ######################
vocab_size = sum(dictionary_bin_1.values())
for k,v in dictionary_bin_1.items():
    dictionary_bin_1[k] = v/vocab_size
    
vocab_size_0 = sum(dictionary_bin_0.values())
for k_0,v_0 in dictionary_bin_0.items():
    dictionary_bin_0[k_0] = v_0/vocab_size_0

In [114]:
########### Creating a dictionary of all the words in each of the binary category 1 and 0 ##############

bin_1_value = X_test.apply(lambda x: \
                           sum([dictionary_bin_1[each] if (each in dictionary_bin_1.keys() and each not in stop_words)\
                                else 0 for each in word_tokenize(x)]))

bin_0_value = X_test.apply(lambda x: \
                           sum([dictionary_bin_0[each] if (each in dictionary_bin_0.keys() and each not in stop_words)\
                                else 0 for each in word_tokenize(x)]))

In [115]:
df_result = pd.DataFrame([bin_1_value,bin_0_value]).T
df_result.columns = ['bin_1_value','bin_0_value']

### 3.3 Language Model Result

In [116]:
(df_result.bin_0_value < df_result.bin_1_value).sum()

1188

###### Accuracy and Distribution

In [117]:
100 * (df_result.bin_0_value < df_result.bin_1_value).sum()/len(df_result.bin_1_value)

35.69711538461539

In [119]:
y_test.value_counts()

0    3197
1     131
Name: Response_binary, dtype: int64

###### Confusion Matrix

In [144]:
from sklearn.metrics import confusion_matrix

preds = (df_result.bin_0_value < df_result.bin_1_value).astype('int')
(preds == y_test).sum()/len(y_test)

0.8792067307692307

In [145]:
confusion_matrix(y_test, preds)

array([[2840,  357],
       [  45,   86]], dtype=int64)

In [121]:
df_result[df_result.bin_0_value < df_result.bin_1_value].head()

,bin_1_value,bin_0_value
16621,0.000255,0.000187
7057,0.017422,0.013946
5704,0.035992,0.032447
14089,0.006956,0.004930
7959,0.008679,0.008282


In [123]:
#### Considering the words with higher probability in 1 and 0

primary_words = []

for each in dictionary_bin_0.keys():
    if dictionary_bin_0[each] < dictionary_bin_1[each]:
        primary_words.append(each)

###### Significant Words

In [125]:
primary_words

['ostensibly',
 'plan',
 'gentle-',
 'California',
 'union',
 'believed',
 'plot',
 'hunt',
 'recognize',
 'blood',
 'mistress',
 'Grass',
 'Valley',
 'coup',
 'celebrity',
 'adventure',
 'Besides',
 'preliminary',
 'expenses',
 'jewellery',
 'Trinity',
 'art',
 'heroine',
 'forsook',
 'offered',
 'permission',
 'struck',
 'Richard',
 'hit',
 'seem',
 'trial',
 'bear',
 'preparing',
 '1837',
 'attached',
 'affection',
 'Aunt',
 'trace',
 'Byron',
 'Moore',
 'drove',
 'Dublin',
 'arms',
 'fashion',
 'gatherings',
 '14',
 'December',
 '23',
 'advanced',
 'odd',
 'beginning',
 'career',
 'destiny',
 'quickly',
 'aside',
 'effect',
 'instrument',
 'talking',
 'Mme',
 'Conti',
 'secret',
 'Court',
 'pierced',
 'actually',
 'faltered',
 'dropped',
 'cheque',
 'Robert',
 'Blue',
 'late',
 'sums',
 'flag',
 'describes',
 'feelings',
 'keys',
 'lover',
 'Bastille',
 'winning',
 "d'Aubray",
 'forgiveness',
 'utterly',
 'happiness',
 'reformed',
 'favour',
 'sup-',
 'wealth',
 'devotion',
 'save'

### 4. Modification - Including the words that are present for the binary class 1

In [127]:
data_bin_1 = X_train[y_train == 1]
data_bin_0 = X_train[y_train == 0]

dict_bin_1_tokens = word_tokenize(' '.join(data_bin_1))
dict_bin_0_tokens = word_tokenize(' '.join(data_bin_0))

dictionary_bin_1 = Counter([each for each in dict_bin_1_tokens if each not in stop_words])
dictionary_bin_0 = Counter([each for each in dict_bin_0_tokens if each not in stop_words and each in dict_bin_1_tokens])

In [128]:
################ Creating a dataframe of probabilites ######################

vocab_size = sum(dictionary_bin_1.values())
for k,v in dictionary_bin_1.items():
    dictionary_bin_1[k] = v/vocab_size
    
vocab_size_0 = sum(dictionary_bin_0.values())
for k_0,v_0 in dictionary_bin_0.items():
    dictionary_bin_0[k_0] = v_0/vocab_size_0

In [129]:
########### Creating a dictionary of all the words in each of the binary category 1 and 0 ##############

bin_1_value = X_test.apply(lambda x: \
                           sum([dictionary_bin_1[each] if (each in dictionary_bin_1.keys() and each not in stop_words)\
                                else 0 for each in word_tokenize(x)]))

bin_0_value = X_test.apply(lambda x: \
                           sum([dictionary_bin_0[each] if (each in dictionary_bin_0.keys() and each not in stop_words)\
                                else 0 for each in word_tokenize(x)]))

In [130]:
df_result = pd.DataFrame([bin_1_value,bin_0_value]).T
df_result.columns = ['bin_1_value','bin_0_value']

### 3.3 Language Model Result

In [131]:
(df_result.bin_0_value < df_result.bin_1_value).sum()

443

In [142]:
y_test.value_counts()

0    3197
1     131
Name: Response_binary, dtype: int64

In [143]:
from sklearn.metrics import confusion_matrix

preds = (df_result.bin_0_value < df_result.bin_1_value).astype('int')
(preds == y_test).sum()/len(y_test)

0.8792067307692307

In [141]:
confusion_matrix(y_test, preds)

array([[2840,  357],
       [  45,   86]], dtype=int64)

In [135]:
df_result[df_result.bin_0_value < df_result.bin_1_value].head()

,bin_1_value,bin_0_value
13407,0.004340,0.002718
4384,0.000383,0.000332
13743,0.006382,0.002522
10356,0.022144,0.015319
16561,0.007020,0.002537


In [136]:
X_test.values[1]

"In strait-laced ways and to all demure modesty, Eliza was reared. And at fifteen she was not only the prettiest girl in Rhode Island, but one of the cleverest andâ\x80\x94so declared the piousâ\x80\x94one of the very worst. In those days and in New England, it was delightfully easy to acquire a reputation for wickedness by merely failing to conform to all the ideas of the blue-law devotees. Shan't we give Betty Bowenâ\x80\x94her com- monly used nameâ\x80\x94the benefit of the doubt?"

In [137]:
#### Considering the words with higher probability in 1 and 0

primary_words = []

for each in dictionary_bin_0.keys():
    if dictionary_bin_0[each] < dictionary_bin_1[each]:
        primary_words.append(each)

In [138]:
primary_words

['ostensibly',
 'gentle-',
 'California',
 'union',
 'plot',
 'hunt',
 'blood',
 'mistress',
 'Grass',
 'Valley',
 'coup',
 'celebrity',
 'adventure',
 'preliminary',
 'expenses',
 'jewellery',
 'Trinity',
 'forsook',
 'permission',
 'Richard',
 'hit',
 'trial',
 'preparing',
 '1837',
 'Aunt',
 'trace',
 'Byron',
 'Moore',
 'drove',
 'Dublin',
 'arms',
 'gatherings',
 'December',
 '23',
 'advanced',
 'odd',
 'destiny',
 'instrument',
 'Conti',
 'secret',
 'Court',
 'pierced',
 'faltered',
 'dropped',
 'cheque',
 'Robert',
 'sums',
 'flag',
 'describes',
 'keys',
 'lover',
 'Bastille',
 'winning',
 "d'Aubray",
 'forgiveness',
 'utterly',
 'reformed',
 'favour',
 'sup-',
 'wealth',
 'devotion',
 'queen',
 'rival',
 'proportions',
 'improbable',
 'arrested',
 'intervened',
 'secure',
 'release',
 'debts',
 'supposing',
 'treatise',
 'extreme',
 'fabric',
 'greatest',
 'exquisite',
 'violent',
 'endure',
 'silence',
 'sincerity',
 'self',
 'conviction',
 'conduct',
 'refusing',
 'recognise